In [3]:
import re
import psutil

from subprocess import check_output

In [4]:
nmcli_output = check_output("nmcli device status", shell=True, encoding="utf-8")

In [5]:
print(re.findall(r".*wlan0.*.wifi.*.connected.*", nmcli_output)[0].split()[3])
print(re.findall(r".*enp0s20f0u3u3.*.ethernet.*.connected.*", nmcli_output) or None)

Vodafone-46EC
None


In [24]:
with open("/sys/class/power_supply/BAT0/charge_full_design") as f:
    capacity = int(f.read())
with open("/sys/class/power_supply/BAT0/charge_now") as f:
    current_charge = int(f.read())
with open("/sys/class/power_supply/BAT0/charge_full") as f:
    current_capacity = int(f.read())

capacity, current_charge, current_capacity, round(current_charge * 100 / capacity, 2)

(3534000, 2126000, 2126000, 60.16)

In [42]:
def hex_to_rgb(hex_value):
    hex_value = hex_value.lstrip('#')
    return f'\033[38;2;{";".join([str(int(hex_value[i:i+2], 16)) for i in (0, 2, 4)])}m'

"""
set $surface1  #45475a
set $pink      #f5c2e7
set $base      #1e1e2e
"""
surface1 = "#45475a"
pink = "#f5c2e7"
base = "#1e1e2e"

COLOR_END = '\033[0m'
PINK = hex_to_rgb(pink)
SURFACE1 = hex_to_rgb(surface1)
BASE = hex_to_rgb(base)

# Example usage with hex values
print(f"{PINK}This is pink text.{COLOR_END}")
print(f"{SURFACE1}This is surface1 text.{COLOR_END}")
print(f"{BASE}This is base text.{COLOR_END}")

This is blue text.
This is blue text.
This is blue text.


In [6]:
print(psutil.sensors_battery())
print([(sensor.current, sensor.label) for sensor in psutil.sensors_temperatures()['dell_ddv'] if sensor.label in ["CPU", "Video", "Ambient"]])

sbattery(percent=95.24929444967074, secsleft=<BatteryTime.POWER_TIME_UNLIMITED: -2>, power_plugged=True)
[(53.0, 'CPU'), (52.0, 'Ambient'), (49.0, 'Video')]


In [9]:
from datetime import datetime
from psutil import disk_usage, sensors_battery, virtual_memory, cpu_stats
from psutil._common import bytes2human
from socket import gethostname, gethostbyname
from subprocess import check_output
from sys import stdout
from time import sleep

def write(data):
    stdout.write('%s\n' % data)
    stdout.flush()

def refresh():
    disk = bytes2human(disk_usage('/').free)
    memory = virtual_memory()
    total_memory = bytes2human(memory.total)
    available_memory = bytes2human(memory.available)
    used_memory = bytes2human(memory.used)
    shared_memory = bytes2human(memory.shared)
    memory = f'{available_memory}:{shared_memory} ({used_memory}/{total_memory})'

    sensors_temperatures = [(sensor.current, sensor.label) for sensor in psutil.sensors_temperatures()['dell_ddv'] if sensor.label in ["CPU", "Video", "Ambient"]]
    cpu_temperatures = ''.join([f'{temp}°C' for temp, label in sensors_temperatures if label == "CPU"])
    video_temperatures = ''.join([f'{temp}°C' for temp, label in sensors_temperatures if label == "Video"])
    ambient_temperatures = ''.join([f'{temp}°C' for temp, label in sensors_temperatures if label == "Ambient"])

    hostname = gethostname()
    nmcli = check_output("nmcli device status", shell=True, encoding="utf-8") 
    wifi = re.findall(r'.*wlan0.*.wifi.*.connected.*', nmcli)[0].split()[3] or None
    ethernet = re.findall(r'.*enp0s20f03u3.*.ethernet.*.connected.*', nmcli) or None
    internet = None

    if ethernet is not None:
        internet += 'Ethernet'
    if wifi is not None and internet is not None:
        internet += f'/{wifi}'
    elif wifi is not None and internet is None:
        internet = wifi
    else:
        internet = "No Internet"

    battery = int(sensors_battery().percent)
    status = "Charging" if sensors_battery().power_plugged else "Discharging"
    date = datetime.now().strftime('%h %d %A %H:%M')
    write(f'Temperatures: {cpu_temperatures} : {video_temperatures} : {ambient_temperatures} | Space: {disk} | Memory: {memory} | Internet: {internet} | Battery: {status} {battery}% | Date: {date} | {hostname}')

refresh()

Temperatures: 52.0°C : 49.0°C : 51.0°C | Space: 143.6G | Memory: 8.0G:650.3M (6.4G/15.4G) | Internet: Vodafone-46EC | Battery: Charging 95% | Date: Feb 25 Sunday 11:54 | kusharchl


In [25]:
import re

from datetime import datetime
from psutil import disk_usage, sensors_battery, virtual_memory, sensors_temperatures
from psutil._common import bytes2human
from subprocess import check_output
from sys import stdout


status_command = []


def write(data):
    stdout.write('%s\n' % data)
    stdout.flush()


def refresh():
    disk = bytes2human(disk_usage('/').free)

    memory = virtual_memory()
    total_memory = bytes2human(memory.total)
    available_memory = bytes2human(memory.available)
    used_memory = bytes2human(memory.used)
    shared_memory = bytes2human(memory.shared)
    memory = f'{available_memory}:{shared_memory} ({used_memory}/{total_memory})'

    with open("/sys/class/power_supply/BAT0/charge_full_design") as f:
        capacity = int(f.read())
    with open("/sys/class/power_supply/BAT0/charge_now") as f:
        current_charge = int(f.read())
    with open("/sys/class/power_supply/BAT0/charge_full") as f:
        current_capacity = int(f.read())
    capacity, current_charge, current_capacity, round(current_charge * 100 / capacity, 2)
    status = None
    battery_plugged = sensors_battery().power_plugged
    if current_capacity == current_charge and battery_plugged:
        status = "FULL PLUGGED"
    elif current_capacity == current_charge and not battery_plugged:
        status = "FULL"
    elif current_charge * 100 / current_capacity < 20 and not battery_plugged:
        status = "LOW"
    elif current_charge * 100 / current_capacity < 20 and battery_plugged:
        status = "LOW CHARGING"
    elif battery_plugged:
        status = "CHARGING"
    else:
        status = "DISCHARGING"
    battery = round(current_charge * 100 / capacity, 2)

    sensor_temperatures = [(sensor.current, sensor.label) for sensor in sensors_temperatures()['dell_ddv'] if sensor.label in ["CPU", "Video", "Ambient"]]
    cpu_temperatures = ''.join([f'{temp}°C' for temp, label in sensor_temperatures if label == "CPU"])
    video_temperatures = ''.join([f'{temp}°C' for temp, label in sensor_temperatures if label == "Video"])
    ambient_temperatures = ''.join([f'{temp}°C' for temp, label in sensor_temperatures if label == "Ambient"])

    nmcli = check_output("nmcli device status", shell=True, encoding="utf-8") 
    wifi = re.findall(r'.*wlan0.*.wifi.*.connected.*', nmcli)[0].split()[3] or None
    ethernet = re.findall(r'.*enp0s20f03u3.*.ethernet.*.connected.*', nmcli) or None
    internet = None

    if ethernet is not None:
        internet += 'Ethernet'
    if wifi is not None and internet is not None:
        internet += f'/{wifi}'
    elif wifi is not None and internet is None:
        internet = wifi
    else:
        internet = "No Internet"
    internet = remove_ansi_escape(internet)

    date = datetime.now().strftime('%h %d %A %H:%M:%S')


    # Theme
    SURFACE1, PINK, BASE = get_colors()
    # END = "</span>"
    END = "\033[0m"
    SEPERATOR = f' {PINK}|{END} '
    write(f'{SURFACE1}{cpu_temperatures}:{video_temperatures}:{ambient_temperatures}{END}{SEPERATOR}{SURFACE1}{disk}{END}{SEPERATOR}{SURFACE1}{memory}{END}{SEPERATOR}{SURFACE1}{internet}{END}{SEPERATOR}{SURFACE1}{status} {battery}%{END}{SEPERATOR}{SURFACE1}{date}{END}')


def hex_to_rgb(hex):
    hex_value = hex.lstrip('#')
    return f'\033[38;2;{";".join([str(int(hex_value[i:i+2], 16)) for i in (0, 2, 4)])}m'


def remove_ansi_escape(string_with_ansi):
    ansi_escape = re.compile(r'\x1b\[[0-9;]*[mK]')  # Match ANSI escape codes
    return ansi_escape.sub('', string_with_ansi)


def get_colors():
    surface1 = hex_to_rgb("#45475a")
    pink = hex_to_rgb("#f5c2e7")
    base = hex_to_rgb("#1e1e2e")
    # surface1 = hex_to_pango_color("#45475a")
    # pink = hex_to_pango_color("#f5c2e7")
    # base = hex_to_pango_color("#1e1e2e")
    return surface1, pink, base

refresh()

53.0°C:50.0°C:52.0°C | 143.6G | 7.9G:624.8M (6.5G/15.4G) | Vodafone-46EC | CHARGING 59.14% | Feb 25 Sunday 12:06:58


In [7]:
"""
nmcli device status
--------------------
➜ nmcli device status
DEVICE           TYPE      STATE                   CONNECTION
wlan0            wifi      connected               Vodafone-46EC
lo               loopback  connected (externally)  lo
br-5eac55d4d58a  bridge    connected (externally)  br-5eac55d4d58a
br-9bc02d4402a8  bridge    connected (externally)  br-9bc02d4402a8
docker0          bridge    connected (externally)  docker0
p2p-dev-wlan0    wifi-p2p  disconnected            --
enp0s20f0u3u3    ethernet  unavailable             --

For WIFI
------------
GENERAL.DEVICE:                         wlan0
GENERAL.TYPE:                           wifi
GENERAL.HWADDR:                         18:47:3D:1A:04:85
GENERAL.MTU:                            1500
GENERAL.STATE:                          100 (connected)
GENERAL.CONNECTION:                     Vodafone-46EC
GENERAL.CON-PATH:                       /org/freedesktop/NetworkManager/ActiveConnection/5
IP4.ADDRESS[1]:                         192.168.0.15/24
IP4.GATEWAY:                            192.168.0.1
IP4.ROUTE[1]:                           dst = 192.168.0.0/24, nh = 0.0.0.0, mt = 600
IP4.ROUTE[2]:                           dst = 0.0.0.0/0, nh = 192.168.0.1, mt = 600
IP4.DNS[1]:                             192.168.0.1
IP6.GATEWAY:                            --

For ethernet
------------
GENERAL.DEVICE:                         enp0s20f0u3u3
GENERAL.TYPE:                           ethernet
GENERAL.HWADDR:                         00:E0:4C:36:1A:91
GENERAL.MTU:                            1500
GENERAL.STATE:                          20 (unavailable)
GENERAL.CONNECTION:                     --
GENERAL.CON-PATH:                       --
WIRED-PROPERTIES.CARRIER:               off
IP4.GATEWAY:                            --
IP6.GATEWAY:                            --
"""

'\nnmcli device status\n--------------------\n➜ nmcli device status\nDEVICE           TYPE      STATE                   CONNECTION\nwlan0            wifi      connected               Vodafone-46EC\nlo               loopback  connected (externally)  lo\nbr-5eac55d4d58a  bridge    connected (externally)  br-5eac55d4d58a\nbr-9bc02d4402a8  bridge    connected (externally)  br-9bc02d4402a8\ndocker0          bridge    connected (externally)  docker0\np2p-dev-wlan0    wifi-p2p  disconnected            --\nenp0s20f0u3u3    ethernet  unavailable             --\n\nFor WIFI\n------------\nGENERAL.DEVICE:                         wlan0\nGENERAL.TYPE:                           wifi\nGENERAL.HWADDR:                         18:47:3D:1A:04:85\nGENERAL.MTU:                            1500\nGENERAL.STATE:                          100 (connected)\nGENERAL.CONNECTION:                     Vodafone-46EC\nGENERAL.CON-PATH:                       /org/freedesktop/NetworkManager/ActiveConnection/5\nIP4.ADDRESS[1